## Document Reviewer with Cohere

### Goals
We want to explore how Cohere perform as a document reviewer. Given a technical documentation, can Cohere generate an improved version of it better than other LLMs?

### Techniques


- all-in-one prompt: we first start by putting all the desired outcome in a single prompt
- multiple prompts by roles: we feed the document into individual prompts that does only one thing. The completion is then fed into another prompt until it goes through all the chain
- decomposition: we ask the LLM to decompose the document first, and then convert the documents content to questions instead. Then we ask the LLM to answer the questions and then reconstruct the document.

### Loading

We first load our sample document to improve.

In [2]:
document = """
# Use Constant


## Avoid const*.go

Avoid naming files with any variation of `const.go` to store constants.

Each constant serves a purpose, and they probably should belong to domain layer.


## Don't uppercase constant


```go
// Bad
const CODE_NOT_FOUND = 404


// Good
const CodeNotFound = 404
```

## Prefer constructor over constant


There are many times where constructor proves to be more useful than plain constant.

For example, you may have a constant to declare the expiry time of a campaign:


```go
var CampaignExpiredAt = time.Date(2023, time.January, 1, 0, 0, 0, 0, time.Local)
```

However, it doesn't serve much purpose there. Instead, add an accompanying function that utilizes it.

```go
package main

import (
	"fmt"
	"time"
)

var CampaignExpiredAt = time.Date(2023, time.January, 1, 0, 0, 0, 0, time.Local)

func CampaignActive(now time.Time) bool {
	return now.Before(CampaignExpiredAt)
}

func main() {
	now := time.Now()
	fmt.Println(now)
	fmt.Println(CampaignActive(now))
}
```


Other examples includes
- base url
- expiry duration
- cache prefix key
"""

In [3]:
from IPython.display import Markdown

In [7]:
Markdown(document)


# Use Constant


## Avoid const*.go

Avoid naming files with any variation of `const.go` to store constants.

Each constant serves a purpose, and they probably should belong to domain layer.


## Don't uppercase constant


```go
// Bad
const CODE_NOT_FOUND = 404


// Good
const CodeNotFound = 404
```

## Prefer constructor over constant


There are many times where constructor proves to be more useful than plain constant.

For example, you may have a constant to declare the expiry time of a campaign:


```go
var CampaignExpiredAt = time.Date(2023, time.January, 1, 0, 0, 0, 0, time.Local)
```

However, it doesn't serve much purpose there. Instead, add an accompanying function that utilizes it.

```go
package main

import (
	"fmt"
	"time"
)

var CampaignExpiredAt = time.Date(2023, time.January, 1, 0, 0, 0, 0, time.Local)

func CampaignActive(now time.Time) bool {
	return now.Before(CampaignExpiredAt)
}

func main() {
	now := time.Now()
	fmt.Println(now)
	fmt.Println(CampaignActive(now))
}
```


Other examples includes
- base url
- expiry duration
- cache prefix key


In [8]:
import app.cohere as cohere

In [9]:
prompt = """
You are an experienced technical writer.
Given a technical documentation, improve the content by elaborating on the key insights.

Document:
{document}
"""

In [10]:
completion = cohere.generate_text(prompt.format(document=document))
completion

' Technical writing aims to provide clear and effective documentation for developers and other technical users. Your initial documentation is a good starting point, but there are ways to improve it to make it even more comprehensive and useful. Here\'s an enhanced version of your documentation with more detailed explanations and examples:\n\n# Effective Use of Constants in Go\n\nConstants are an integral part of many programming languages, including Go. They provide a way to define fixed values that should not be changed during program execution. Understanding when and how to use constants effectively can improve code readability and maintainability.\n\n## Avoid Naming Files `const.go`\n\nIt\'s worth emphasizing the importance of avoiding filename variations like `const.go` for storing constants. Filenames should provide meaningful information about the content within, and using a generic name like `const.go` does not offer any insight into the specific constants defined within the fil

In [11]:
Markdown(completion)

 Technical writing aims to provide clear and effective documentation for developers and other technical users. Your initial documentation is a good starting point, but there are ways to improve it to make it even more comprehensive and useful. Here's an enhanced version of your documentation with more detailed explanations and examples:

# Effective Use of Constants in Go

Constants are an integral part of many programming languages, including Go. They provide a way to define fixed values that should not be changed during program execution. Understanding when and how to use constants effectively can improve code readability and maintainability.

## Avoid Naming Files `const.go`

It's worth emphasizing the importance of avoiding filename variations like `const.go` for storing constants. Filenames should provide meaningful information about the content within, and using a generic name like `const.go` does not offer any insight into the specific constants defined within the file.

## Don't Use Uppercase for Constants

While uppercase letters are commonly used to define constants in many languages, including Go, it's good practice to avoid using uppercase for constant names entirely. This is because uppercase letters can make your code less readable and may not align with the style conventions of your project or team.

### Example

Here's an example that demonstrates the difference between using uppercase and lowercase for constant names:

**Bad Practice - Using uppercase:**
```go
const CODE_NOT_FOUND = 404
```

**Good Practice - Using lowercase:**
```go
const CodeNotFound = 404
```

## Prefer Constructors over Constants for Expiry Time

Your documentation rightly suggests that using a constructor function can be more beneficial than a constant for scenarios like declaring campaign expiry time. Constructors can encapsulate logic and provide a more meaningful way to work with values.

### Example: Campaign Expiry Constructor

```go
package main

import (
	"time"
)

func NewCampaignExpiry(year int, month time.Month, day int) time.Time {
	return time.Date(year, month, day, 0, 0, 0, 0, time.Local)
}

func main() {
	expiryTime := NewCampaignExpiry(2023, time.January, 1)
	fmt.Println(expiryTime)
}
```

In this example, the `NewCampaignExpiry` constructor takes parameters for year, month, and day, and it returns a `time.Time` value representing the campaign expiry date. This approach provides more context and allows for easier modification in the future.

## Consider Constructors for Other Use Cases

Your documentation mentions other examples like base URL, expiry duration, and cache prefix key, where constructors could be more appropriate. Here's how constructors could be applied to these examples:

**Base URL:**
Instead of declaring a constant like `const BaseURL = "https://example.com"`, you can use a constructor to create URLs based on different parameters.

**Expiry Duration:**
If you have a constant like `const ExpiryDuration = time.Hour`, it might not be clear what this constant represents. Instead, you could use a constructor to create a specific expiry duration based on required parameters.

**Cache Prefix Key:**
Instead of a constant like `const CachePrefix = "CACHE_"`, you could define a constructor that generates a cache key based on input parameters.

## Example: Cache Prefix Key Constructor

```go
package main

import (
	"fmt"
)

func NewCacheKey(userID int, username string) string {
	return fmt.Sprintf("CACHE_%d_%s", userID, username)
}

func main() {
	cacheKey := NewCacheKey(123, "john_doe")
	fmt.Println(cacheKey)
}
```

In this example, the `NewCacheKey` constructor takes a user ID and a username as parameters and generates a cache key. This approach makes your code more flexible and adaptable to changes.

Remember, these suggestions are based on the assumption that your documentation aims to provide guidance on effective usage of constants in Go. If your documentation has a different focus, the content might be tailored accordingly. 

Would you like me to further elaborate on any of the examples provided? 

In [12]:
prompt = """
You are an experienced technical writer.
Given a technical documentation, improve the content by elaborating on the key insights.
Keep the code samples unchanged. Return only the improved document.

Document:
{document}
"""

completion = cohere.generate_text(prompt.format(document=document))
completion

' Here is the improved version of your document with more detailed explanations of the key insights:\n\n# Understanding When to Use Constants in Go\n\nWhen writing code in Go, it is important to recognize when to use constants and when to use other tools, such as constructors. While constants can be useful in certain situations, there are other scenarios where using a constructor may be more beneficial.\n\n## Avoid Naming Files `const.go`\n\nIt is recommended to avoid naming files in the format of `const.go` when storing constants in Go. Constants should belong to the domain layer of your code, indicating they are fundamental to the application and often used across different packages.\n\n## Don\'t Use Uppercase for Constants\n\nIt is a common convention in Go to use uppercase letters for constants. While this is allowed, it is not recommended because it can make your code less readable and harder to understand, especially when the constant names are used extensively. \n\nInstead, use 

In [13]:
Markdown(completion)

 Here is the improved version of your document with more detailed explanations of the key insights:

# Understanding When to Use Constants in Go

When writing code in Go, it is important to recognize when to use constants and when to use other tools, such as constructors. While constants can be useful in certain situations, there are other scenarios where using a constructor may be more beneficial.

## Avoid Naming Files `const.go`

It is recommended to avoid naming files in the format of `const.go` when storing constants in Go. Constants should belong to the domain layer of your code, indicating they are fundamental to the application and often used across different packages.

## Don't Use Uppercase for Constants

It is a common convention in Go to use uppercase letters for constants. While this is allowed, it is not recommended because it can make your code less readable and harder to understand, especially when the constant names are used extensively. 

Instead, use lowercase names for your constants, just like any other variable name.

## Prefer Constructor over Constant for Expiry Time

Consider a situation where you want to declare the expiry time of a campaign. You could declare a constant for this purpose:

```go
var CampaignExpiredAt = time.Date(2023, time.January, 1, 0, 0, 0, 0, time.Local)
```

However, simply declaring a constant does not provide much functionality. In this case, it would be more useful to create a function that checks if the campaign has expired, using the constant as an argument.

```go
package main

import (
	"fmt"
	"time"
)

var CampaignExpiredAt = time.Date(2023, time.January, 1, 0, 0, 0, 0, time.Local)

func CampaignActive(expiryTime time.Time) bool {
	return expiryTime.Before(CampaignExpiredAt)
}

func main() {
	now := time.Now()
	fmt.Println(now)
	fmt.Println(CampaignActive(now))
}
```

This way, the constant is used as an argument for the `CampaignActive` function, which provides more utility and understanding to other developers reading your code.

Other examples where constants could be used include:
- Base URL: It can be helpful to define a constant for the base URL of your application to prevent hard-coding throughout your code. 

While constants are a valuable tool in certain situations, it is important to recognize when they are not the best solution. In these examples, using a constant instead of a constructor or variable could lead to less readable and less functional code. 

Remember, the key to good coding practices is to maintain readability and functionality, and sometimes this means using a tool other than constants, even if they are a valid option. 

Would you like me to help you with anything else regarding technical writing or Go code? 

In [22]:
prompt = """
You are provided a technical document. Think step by step.
1. extract the key topic of this document. 
2. suggest points about this topic that is not already present in the document
3. ask critical and relevant questions about the topics
4. attempt to answer the questions about the topic, providing clear and simple explanation.

Summarize all of it into an improved version of the document. Return only the improved version in Markdown.

Document:
----------------------------------------
{document}
----------------------------------------
"""

completion = cohere.generate_text(prompt.format(document=document))
completion

" Based on the document, it seems the key topic is on when and how to use constants in Go programming, and when it might be better to use a constructor or variable instead. \n\nHere are some points that could be added to the document to improve its coverage on the topic:\n\n1. **Importance of constants**: Explain why constants are useful in programming and how they differ from variables. For example, you could mention that constants provide a way to define a value that shouldn't be changed during program execution, which can make code more readable and maintainable.\n\n2. **When to use constants**: Provide more context on when it would be appropriate to use a constant instead of a variable. For example, you could mention that constants are useful for values that are set once and won't change, like status codes, magic numbers, or dimension constants.\n\n3. **Importance of consistency**: Explain why it's important to be consistent with your use of constants, including naming conventions 

In [23]:
Markdown(completion)

 Based on the document, it seems the key topic is on when and how to use constants in Go programming, and when it might be better to use a constructor or variable instead. 

Here are some points that could be added to the document to improve its coverage on the topic:

1. **Importance of constants**: Explain why constants are useful in programming and how they differ from variables. For example, you could mention that constants provide a way to define a value that shouldn't be changed during program execution, which can make code more readable and maintainable.

2. **When to use constants**: Provide more context on when it would be appropriate to use a constant instead of a variable. For example, you could mention that constants are useful for values that are set once and won't change, like status codes, magic numbers, or dimension constants.

3. **Importance of consistency**: Explain why it's important to be consistent with your use of constants, including naming conventions and placement. Consistency can improve code readability and maintainability.

4. **Places to use constants in Go**: Provide more details on common places to use constants in Go code, besides the ones already mentioned like base URL, expiry duration and cache prefix key. For example, you could mention using constants for configuration values, or defining codes for different application states.

5. **When to avoid constants**: Explain scenarios where using a constant could be detrimental to the code, and when it might be better to use a variable or a constructor. For example, you could mention that using a constant for a value that may need to change during runtime could lead to code that's harder to maintain.

Here are some critical questions about the topic that the document doesn't answer:

1. **When is it appropriate to use a constant instead of a variable in Go, from a technical perspective?**
2. **How does the performance of a constant vs a variable compare in Go, and how might this affect the programmer's decision on when to use each?**
3. **What are some advanced use cases for constants in Go that demonstrate their flexibility?**

And here are some attempts to answer these questions:

1. **When is it appropriate to use a constant instead of a variable in Go, from a technical perspective?**
   Constants in Go are immutable values that are set once and cannot be changed during the execution of the program. This makes them ideal for values that are known at compile time and remain constant throughout the program's execution. 

Variables, on the other hand, are dynamic entities that can be assigned new values at runtime. They are suitable for storing values that may change over the course of the program. 

Therefore, it is appropriate to use a constant when you have a value that is set once and will not change, and a variable when you have a value that may change over time. 

From a technical perspective, constants can improve code performance because the compiler can optimize constant values that are known at compile time. 

2. **How does the performance of a constant vs a variable compare in Go, and how might this affect the programmer's decision on when to use each?**
   In general, the performance difference between using constants and variables in Go is minimal and unlikely to significantly impact your decision. 

The Go compiler can potentially optimize constant expressions by performing constant folding, which can lead to some performance improvements. However, this is not guaranteed and highly dependent on the specific usage and compiler optimizations.

For most cases, it would be more important to consider other factors like code readability, maintainability, and flexibility when deciding between using constants or variables. 

3. **What are some advanced use cases for constants in Go that demonstrate their flexibility?**
   Constants in Go can be used in various advanced scenarios. Here are a few examples:
- **Constant Masks**: You can create bit masks using constants in Go, which can be useful for performing bitwise operations or extracting specific bits from an integer.

- **Constant Variables**: Although Go does not have constant variables in the same way that some other languages do, you can use variables initialized with constants to achieve a similar effect. This can be helpful when you need to perform operations on constant values but still want to maintain the flexibility of a variable.

- **Constant Functions**: You can define functions with constant values as parameters or return values, which can be useful for encapsulating logic that operates on specific constant values.

Overall, the flexibility of constants in Go allows them to be used in various advanced scenarios beyond simple value assignments. 

4. **When is it better to use a constructor or variable instead of a constant?**
   There are times when using a constructor or variable may be more appropriate than using a constant. Here are a few scenarios:

- **Dynamic Values**: If the value you need to work with can change over time or may have different values based on specific conditions, using a variable would provide more flexibility. A constructor can be used to initialize the variable with different values based on certain conditions or user input.

- **Mutable State**: If you need to maintain a mutable state that can be updated over time, using a variable would allow you to modify its value without redefining the constant. A constructor could be used to set the initial state of the variable.

- **Configuration at Runtime**: If your application requires dynamic configuration changes at runtime, using variables and constructors would provide the flexibility to adjust these settings without the need to redefine constants. 

Remember that these are general guidelines, and the specific choice between constants, constructors, and variables will depend on the context and requirements of your code. 

I hope these additional points and answers are helpful in improving the document and providing a more comprehensive understanding of the topic. 

In [25]:
prompt = """
You are given summaries of a technical document on a specific topic.
Write an RFC about this topic based on the provided summary. 
Return the document as a Markdown:

Summaries:
{summary}
"""
completion = cohere.generate_text(prompt.format(summary=completion))
completion

' RFC: Guidelines for the Use of Constants in Go\n\nDear Go Developers,\n\nAs programmers, we are constantly looking for ways to write clean, efficient, and maintainable code. One fundamental concept that can help us achieve this is the appropriate use of constants. In this RFC, we will discuss the importance of constants in Go programming, provide guidelines on when and where to use them, and explore advanced use cases that demonstrate their flexibility.\n\n1. Importance of Constants\n   Constants are an essential tool in programming languages like Go, providing a way to define immutable values that should not be changed during the execution of a program. Here are some key reasons why constants are important:\n   - Readability: Constants improve code readability by providing meaningful names to values that are used throughout the code. It\'s easier to understand the intent of the code when variables are labeled with self-explanatory constants rather than arbitrary values.\n\n   - Main

In [26]:
Markdown(completion)

 RFC: Guidelines for the Use of Constants in Go

Dear Go Developers,

As programmers, we are constantly looking for ways to write clean, efficient, and maintainable code. One fundamental concept that can help us achieve this is the appropriate use of constants. In this RFC, we will discuss the importance of constants in Go programming, provide guidelines on when and where to use them, and explore advanced use cases that demonstrate their flexibility.

1. Importance of Constants
   Constants are an essential tool in programming languages like Go, providing a way to define immutable values that should not be changed during the execution of a program. Here are some key reasons why constants are important:
   - Readability: Constants improve code readability by providing meaningful names to values that are used throughout the code. It's easier to understand the intent of the code when variables are labeled with self-explanatory constants rather than arbitrary values.

   - Maintainability: By using constants, you can update the value in one place if it ever needs to be changed. This reduces the chance of introducing errors in multiple places throughout the code. It also helps in debugging, as you can easily analyze the impact of changes by looking at the constant's usage.

   - Optimization: In some cases, compilers can optimize code when constants are used instead of variables. Since constants have fixed values, the compiler can perform certain optimizations, such as constant folding, which may result in improved execution speed.

2. When to Use Constants
   Constants should be used when you have values that are known at compile time and remain constant throughout the execution of the program. Here are some specific scenarios where using constants is recommended:
   - Status Codes: Defining status codes or error codes as constants makes it clear to readers what kind of response a function might return.

   - Magic Numbers: Using constants instead of hard-coding numeric values (often referred to as "magic numbers") makes the code more readable and maintainable.

   - Dimension Constants: Constants can be used to define dimensions or sizes, such as the number of items in a list or the dimensions of a grid.

3. Consistency
   It is important to adhere to consistent naming conventions and placement of constants throughout your code. This improves code readability and maintainability for yourself and other developers working on the project.

4. Places to Use Constants in Go
   Besides the examples mentioned earlier, there are several other common places to use constants in Go:
   - Configuration Values: Constants can be used to define configuration settings for an application, making it easier to adjust settings without modifying code.

   - Base URLs: Defining constants for base URLs can make it easier to construct URLs for different endpoints in your application.

   - Time Related Constants: Constants can be used to define durations or timeouts, such as cache expiration durations or intervals between task executions.

5. When to Avoid Constants
   While constants are useful, there are situations where using them might lead to code that is harder to maintain or less flexible. Here are scenarios where using a constant might not be the best choice:
   - Dynamic Values: If a value may need to change during runtime based on user input or other dynamic factors, a variable would provide more flexibility.

   - Mutable State: If you need to maintain a mutable state that changes over time, using a variable would be more appropriate. 

Critical Questions Answered

1. When is it appropriate to use a constant instead of a variable in Go, from a technical perspective?
   Constants in Go are immutable values that are set once and remain constant throughout the program's execution. They are suitable for values that are known at compile time and don't change. Variables, on the other hand, are dynamic and can be assigned new values at runtime, making them suitable for values that may change over time. 

2. How does the performance of a constant vs a variable compare in Go, and how might this affect the programmer's decision on when to use each?
   The performance difference between constants and variables in Go is minimal. While the Go compiler can potentially optimize constant expressions through constant folding, this optimization is not guaranteed and may vary depending on specific usage and compiler optimizations. For most cases, it's more important to consider factors like code readability and maintainability when deciding between constants and variables. 

3. What are some advanced use cases for constants in Go that demonstrate their flexibility?
   Constants in Go can be used in various advanced scenarios. Some examples include:
- Constant Masks: Constants can be used to create bit masks, which are useful for performing bitwise operations or extracting specific bits from an integer.

- Constant Variables: Although Go does not have constant variables like some other languages, you can use variables initialized with constants to achieve a similar effect. This can be helpful when you need to perform operations on constant values while still maintaining flexibility.

- Constant Functions: You can define functions with constant values as parameters or return values, useful for encapsulating logic that operates on specific constant values.

4. When is it better to use a constructor or variable instead of a constant?
   There are scenarios where using a constructor or variable may provide more flexibility than a constant. Some examples include:
- Dynamic Values: If the value you are working with can change over time or may have different values based on specific conditions, using a variable would provide more flexibility. 

5. Summary
   Constants are an important tool in writing clean and maintainable code. They provide a way to define immutable values that improve code readability and can be optimized by the compiler. By following the guidelines outlined in this RFC, developers can make informed decisions about when and where to use constants in their Go programs, and leverage their flexibility in more advanced use cases. 

We encourage all Go developers to consider these guidelines and to share their thoughts and experiences with using constants in the comments section below. Together, we can continue to improve the Go programming language and the ecosystem around it. 

Best regards,

The Go Community

(Edited by Chatbot)

Would you like me to generate a draft of the email intended to send this RFC to the Go Community? 